In [1]:
library(tidyverse)
library(gbm)
#install.packages("caret")
library(caret)
library(pROC)
#install.packages("kernlab")
library(kernlab)

#install.packages("splitTools")
library(splitTools)

#install.packages('multiROC')
library(multiROC)

#install.packages('dummies')
library(dummies)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ───────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ──────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loaded gbm 2.1.8
Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following objects are masked from ‘package:stats’:

    cov, smooth, var


Attaching package: ‘kernlab’

The following object is masked from ‘package:purrr’:

    cross



In [2]:
#ATAC_pred <-  readRDS(snakemake@input[["input_predictions"]])
ATAC_pred <- readRDS("../data/ATAC_predictions_train_20_predict_80/ATAC_pred_lasso_normalized_trimmed_formatted_standardized.rds")

head(ATAC_pred)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

,sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr9_39,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399
V1,PGDX16568P,0.7621097,-0.8032188,0.6402487,-0.8688929,-0.4792134,-0.5096581,0.02594817,0.7295906,0.7738027,...,-0.405123632,0.59187320,-0.29125018,-0.58276010,0.3792968,-0.04366948,0.9183014,0.7664238,0.9980457,-1.0572778
V2,PGDX16569P,0.7057109,-0.8227554,0.7868749,-1.0407754,-0.9284673,-0.4139074,-0.16594560,0.7765621,0.5761691,...,-0.849313481,0.39431325,0.05197526,-0.36318168,0.1744439,-0.13785162,0.5392077,0.3942941,1.7972449,-0.3619464
V3,PGDX16570P,0.4891340,-1.1448217,0.9478602,-1.0930696,-0.5115270,-0.7552269,-0.48824613,0.9537215,0.6624865,...,-0.871941533,-0.06389996,-0.22907645,-0.32470631,-0.6251958,0.17051719,0.6882020,0.1744819,1.4357406,-0.2330366
V4,PGDX16571P,0.6882727,-0.8585679,0.6248431,-0.8310281,-0.5335804,-0.5125528,-0.34185944,0.9281618,0.4768828,...,-0.005893625,0.18185487,0.32387927,0.04577069,0.2752394,0.22750427,0.6333635,0.8413436,1.2819797,0.5629469
V5,PGDX16579P,0.5801896,-1.1674832,0.7883376,-1.0498406,-1.2475692,-0.7345196,-0.18778863,0.2546093,0.2920616,...,-0.067895629,-0.05197828,-0.18308631,-0.70242368,-0.4281903,-0.06235782,1.0764914,1.1411410,0.8795170,-0.4910720
V6,PGDX16580P,0.6831409,-0.8712906,0.7199476,-1.1853489,-0.8520159,-0.6829992,-0.35499242,0.8598551,0.3870744,...,-0.735310376,0.36325245,0.20814984,-0.50307962,0.2609116,0.53933999,1.0772367,0.6906960,1.7696570,-0.4181884


In [3]:
#sample_types <- read.table(snakemake@input[["input_sample_types"]], header = F, sep = " ")
sample_types <- read.table("../data/sample_types.txt", header = F, sep = " ")

colnames(sample_types) <- c("sample", "sample_type")
head(sample_types)

sample,sample_type
PGDX10344P1,Gastric_cancer
PGDX10346P1,Gastric_cancer
PGDX10349P1,Gastric_cancer
PGDX10351P1,Gastric_cancer
PGDX10566P,Ovarian_Cancer
PGDX10567P,Ovarian_Cancer


In [4]:
data <- merge(ATAC_pred, sample_types, by="sample")
#data <- data %>% mutate(sample_type01 = ifelse(sample_type == "Healthy", "Healthy", "Cancer"))

head(data)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399,sample_type
PGDX10344P1,0.28476325,-0.8212573,1.0277500,-0.7094983,-0.2327985,-0.7836319,0.01979819,0.2624251,0.3128992,...,-0.34089350,0.006860094,-0.5010935,-0.2043651,0.02437101,0.57938365,-0.4136060,1.3160044,-0.26256223,Gastric_cancer
PGDX10346P1,1.28471727,-0.8295661,1.0145149,-0.6213053,-0.8143027,-0.6479755,-0.06975614,0.9264024,0.4895216,...,-0.17629217,-0.421600296,-0.5810840,0.1408379,0.21122926,1.13429106,0.8116547,0.3187471,-0.48265821,Gastric_cancer
PGDX10349P1,0.07110482,-1.0079115,0.3347017,-1.1303365,-0.8177276,-0.9202015,-0.08484151,0.5183954,2.2808829,...,1.22409122,0.527804492,-0.1435833,0.4967880,0.15991730,0.89522351,0.5305742,1.6767953,-0.02839009,Gastric_cancer
PGDX10351P1,1.36298501,-1.4848879,0.4225454,-1.3192129,-0.7097136,-0.4653093,-0.42147744,0.9729417,0.5944980,...,0.43571295,0.872833915,-0.0670184,0.7394425,0.13690205,1.29732717,0.4739056,1.4153912,-0.12217374,Gastric_cancer
PGDX10566P,0.40245603,-1.0782760,1.2759516,-0.8646247,-0.5192562,-0.8555661,-0.22999396,0.7699482,0.4825674,...,0.04951928,0.173364771,-0.8705736,0.4935296,-0.17648798,-0.02395699,0.3182328,1.3930824,-0.47078676,Ovarian_Cancer
PGDX10567P,0.97825189,-0.6787169,1.0934262,-0.1749540,-0.6319351,-0.7585842,0.22653017,0.5994301,0.6407291,...,0.50968208,-0.241967372,-0.6951369,0.2412607,0.11914801,0.07982386,0.2854560,1.7946488,-0.69550001,Ovarian_Cancer


In [5]:
data %>% group_by(sample_type) %>% summarize(n = n())
data <-subset(data, sample_type != "Duodenal_Cancer")
data$sample_type <- as.factor(data$sample_type)
data <- data %>% droplevels("Duodenal_Cancer")
data %>% group_by(sample_type) %>% summarize(n = n())

sample_type,n
Bile_Duct_Cancer,25
Breast_Cancer,54
Colorectal_Cancer,26
Duodenal_Cancer,1
Gastric_cancer,27
Healthy,194
Lung_Cancer,35
Ovarian_Cancer,28
Pancreatic_Cancer,34


sample_type,n
Bile_Duct_Cancer,25
Breast_Cancer,54
Colorectal_Cancer,26
Gastric_cancer,27
Healthy,194
Lung_Cancer,35
Ovarian_Cancer,28
Pancreatic_Cancer,34


In [6]:
data <- data %>% select(-sample)
head(data)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,chr10_407,...,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399,sample_type
0.28476325,-0.8212573,1.0277500,-0.7094983,-0.2327985,-0.7836319,0.01979819,0.2624251,0.3128992,1.45146500,...,-0.34089350,0.006860094,-0.5010935,-0.2043651,0.02437101,0.57938365,-0.4136060,1.3160044,-0.26256223,Gastric_cancer
1.28471727,-0.8295661,1.0145149,-0.6213053,-0.8143027,-0.6479755,-0.06975614,0.9264024,0.4895216,0.04814456,...,-0.17629217,-0.421600296,-0.5810840,0.1408379,0.21122926,1.13429106,0.8116547,0.3187471,-0.48265821,Gastric_cancer
0.07110482,-1.0079115,0.3347017,-1.1303365,-0.8177276,-0.9202015,-0.08484151,0.5183954,2.2808829,1.33462998,...,1.22409122,0.527804492,-0.1435833,0.4967880,0.15991730,0.89522351,0.5305742,1.6767953,-0.02839009,Gastric_cancer
1.36298501,-1.4848879,0.4225454,-1.3192129,-0.7097136,-0.4653093,-0.42147744,0.9729417,0.5944980,0.97671516,...,0.43571295,0.872833915,-0.0670184,0.7394425,0.13690205,1.29732717,0.4739056,1.4153912,-0.12217374,Gastric_cancer
0.40245603,-1.0782760,1.2759516,-0.8646247,-0.5192562,-0.8555661,-0.22999396,0.7699482,0.4825674,0.76308429,...,0.04951928,0.173364771,-0.8705736,0.4935296,-0.17648798,-0.02395699,0.3182328,1.3930824,-0.47078676,Ovarian_Cancer
0.97825189,-0.6787169,1.0934262,-0.1749540,-0.6319351,-0.7585842,0.22653017,0.5994301,0.6407291,0.64706533,...,0.50968208,-0.241967372,-0.6951369,0.2412607,0.11914801,0.07982386,0.2854560,1.7946488,-0.69550001,Ovarian_Cancer


In [7]:
cross_validation <- function(dataset, k_inner_cv, k_outer_cv){
    
    observed_all  <- dataset$sample_type
    classes <- unique(observed_all)
    return_tibble <- tibble(observed = rep(observed_all, k_outer_cv), 
                            CV_rep = rep(1:k_outer_cv, each=nrow(dataset)))
    
     for(class in 1:length(unique(observed_all))){
        observed <- ifelse(observed_all==classes[class], as.character(classes[class]), "other")
        return_vector_for_class <- c()
        
        for (i in 1:k_outer_cv){ # repeated Cross-validation loop

            set.seed(i)
            #cvfolds <- cut(seq_len(nrow(dataset)), breaks = k_inner_cv, labels = F)
            #cvfolds <- sample(cvfolds)

            folds <- create_folds(observed, k = k_inner_cv)
            predicted <- rep(NA, nrow(dataset))

            for (fold in folds){
                #rows      <- which(cvfolds==n)
                testdata  <- dataset[-fold,]
                testdata <- testdata %>% select(-sample_type)

                traindata <- dataset[fold,]
                traindata$sample_type <- ifelse(traindata$sample_type==classes[class], as.character(classes[class]), "other")

                ################# Nested cross validation #######################
                set.seed(0)
                seeds <- vector(mode = "list", length = 11)
                for(i in 1:10) seeds[[i]]<- sample.int(n=2000, 1200)
                #for the last model
                seeds[[11]]<-sample.int(1000, 1)

                trControl_svm <- trainControl(method = "repeatedcv", 
                                              seeds = seeds,
                                              number = 3, 
                                              repeats = 1, 
                                              classProbs = TRUE,
                                              verboseIter = ifelse(is.null(getOption('knitr.in.progress')), TRUE, FALSE))

                #svmGrid <- expand.grid(C = c(0.01, 0.1, 0.5, 1, 2, 3, 4, 5, 6, 7))

                fit <- train(sample_type ~ .,
                             data = traindata, 
                             method = "svmPoly",
                             tuneLength = 2,
                             trControl = trControl_svm,
                             preProc = c("center", "scale"),
                             verbose=F)

                #message(plot(fit))
                #message("besttune")
                #message(fit$bestTune)
                #message("fit")
                #message(fit)
                #################################################################

                fitControl <- trainControl(classProbs = TRUE)
                fit2 <- train(sample_type ~ .,
                             data = traindata,
                             method =  "svmPoly",
                             trControl = fitControl,
                             verbose = FALSE,
                             tuneGrid = data.frame(C = fit$bestTune$C, 
                                                   degree = fit$bestTune$degree, 
                                                   scale = fit$bestTune$scale),
                             preProc = c("center", "scale"))
                
                IRdisplay::display(predict(fit2, newdata = testdata, type = "prob"))
                tmp <- predict(fit2, newdata = testdata, type = "prob")[,1]
                IRdisplay::display(tmp)
                predicted[-fold] <- tmp
                }

            return_vector_for_class <- c(return_vector_for_class, predicted)
            } # end of outer cv loop

    return_tibble <- cbind(return_tibble, tibble("{classes[class]}_pred" := return_vector_for_class))
    }
    return(return_tibble)
}

In [ ]:
k_outer_cv = 2
results <- cross_validation(data, k_inner_cv = 2, k_outer_cv = k_outer_cv)

+ Fold1.Rep1: degree=1, scale=0.001, C=0.25 
- Fold1.Rep1: degree=1, scale=0.001, C=0.25 
+ Fold1.Rep1: degree=2, scale=0.001, C=0.25 
- Fold1.Rep1: degree=2, scale=0.001, C=0.25 
+ Fold1.Rep1: degree=1, scale=0.010, C=0.25 
- Fold1.Rep1: degree=1, scale=0.010, C=0.25 
+ Fold1.Rep1: degree=2, scale=0.010, C=0.25 
- Fold1.Rep1: degree=2, scale=0.010, C=0.25 
+ Fold1.Rep1: degree=1, scale=0.001, C=0.50 
- Fold1.Rep1: degree=1, scale=0.001, C=0.50 
+ Fold1.Rep1: degree=2, scale=0.001, C=0.50 
- Fold1.Rep1: degree=2, scale=0.001, C=0.50 
+ Fold1.Rep1: degree=1, scale=0.010, C=0.50 
- Fold1.Rep1: degree=1, scale=0.010, C=0.50 
+ Fold1.Rep1: degree=2, scale=0.010, C=0.50 
- Fold1.Rep1: degree=2, scale=0.010, C=0.50 
+ Fold2.Rep1: degree=1, scale=0.001, C=0.25 
- Fold2.Rep1: degree=1, scale=0.001, C=0.25 
+ Fold2.Rep1: degree=2, scale=0.001, C=0.25 
- Fold2.Rep1: degree=2, scale=0.001, C=0.25 
+ Fold2.Rep1: degree=1, scale=0.010, C=0.25 
- Fold2.Rep1: degree=1, scale=0.010, C=0.25 
+ Fold2.Re

Gastric_cancer,other
0.07133580,0.9286642
0.03842345,0.9615765
0.06162029,0.9383797
0.05984972,0.9401503
0.02813637,0.9718636
0.07235007,0.9276499
0.05412895,0.9458710
0.04796758,0.9520324
0.06119919,0.9388008
0.07713448,0.9228655


[1] 0.07133580 0.03842345 0.06162029 0.05984972 0.02813637 0.07235007
  [7] 0.05412895 0.04796758 0.06119919 0.07713448 0.05215382 0.06225876
 [13] 0.03799834 0.03197825 0.06674118 0.09802225 0.09898626 0.17748390
 [19] 0.02920808 0.08913571 0.08213134 0.08035197 0.07410087 0.05761453
 [25] 0.04200617 0.07673446 0.05359889 0.05991364 0.06314205 0.06771260
 [31] 0.07737399 0.06318089 0.09345025 0.08720915 0.13938587 0.05447901
 [37] 0.06862534 0.03543955 0.05384469 0.06899037 0.05094102 0.04041518
 [43] 0.04557699 0.05695946 0.03383564 0.04463317 0.04975078 0.08249401
 [49] 0.06137577 0.06350719 0.08041388 0.07177726 0.05776723 0.06490405
 [55] 0.05348647 0.07492277 0.03401640 0.06570600 0.04448851 0.06621897
 [61] 0.05886499 0.05411010 0.06349968 0.04163690 0.05410022 0.07059786
 [67] 0.09488389 0.05528429 0.04714939 0.05668924 0.07772164 0.05482833
 [73] 0.06421308 0.06175271 0.04643263 0.06184302 0.07171540 0.08780742
 [79] 0.07148583 0.06827648 0.08936485 0.05017454 0.05348653 0.07293503
 [85] 0.05017306 0.10373675 0.05628304 0.05408785 0.05422873 0.05473999
 [91] 0.05866811 0.07235768 0.05599212 0.06353908 0.05763671 0.06016735
 [97] 0.08758660 0.05965813 0.06050201 0.07160550 0.04853787 0.05949937
[103] 0.07909064 0.07184923 0.06329814 0.09104479 0.07370327 0.07813093
[109] 0.07272783 0.05066667 0.05553960 0.06213824 0.04685870 0.05546166
[115] 0.05821311 0.04224256 0.07268491 0.04834250 0.06908276 0.10371232
[121] 0.05368360 0.06502724 0.08097263 0.08456366 0.06163835 0.08084669
[127] 0.07961657 0.04865125 0.07150013 0.07064408 0.05966983 0.06086363
[133] 0.08109023 0.06170363 0.03938788 0.05744267 0.06452701 0.05179600
[139] 0.04232728 0.06793459 0.03209499 0.04957946 0.10291152 0.03815560
[145] 0.06333740 0.07016792 0.06835976 0.07153819 0.07678183 0.03553222
[151] 0.05780568 0.04422436 0.03936719 0.03861164 0.05589860 0.06388732
[157] 0.06948615 0.05184296 0.06370956 0.05147490 0.05318692 0.04287097
[163] 0.04213783 0.05974603 0.04557729 0.04417032 0.07154087 0.04059037
[169] 0.04267238 0.04572138 0.02894312 0.05445910 0.07576122 0.04263753
[175] 0.01971598 0.04549652 0.05665933 0.05653569 0.07253356 0.11017549
[181] 0.05550419 0.03928317 0.06226096 0.03888171 0.06197357 0.06564864
[187] 0.05202924 0.16429886 0.07945431 0.06255607 0.05968185 0.06016542
[193] 0.07382570 0.06957380 0.03159948 0.08132650 0.07073284 0.04499521
[199] 0.06948611 0.03846119 0.03241377 0.03548323 0.04594757 0.05265392
[205] 0.05770508 0.05903179 0.10178966 0.08122491 0.06194256 0.04738262
[211] 0.01695918

+ Fold1.Rep1: degree=1, scale=0.001, C=0.25 
- Fold1.Rep1: degree=1, scale=0.001, C=0.25 
+ Fold1.Rep1: degree=2, scale=0.001, C=0.25 
- Fold1.Rep1: degree=2, scale=0.001, C=0.25 
+ Fold1.Rep1: degree=1, scale=0.010, C=0.25 
- Fold1.Rep1: degree=1, scale=0.010, C=0.25 
+ Fold1.Rep1: degree=2, scale=0.010, C=0.25 
- Fold1.Rep1: degree=2, scale=0.010, C=0.25 
+ Fold1.Rep1: degree=1, scale=0.001, C=0.50 
- Fold1.Rep1: degree=1, scale=0.001, C=0.50 
+ Fold1.Rep1: degree=2, scale=0.001, C=0.50 
- Fold1.Rep1: degree=2, scale=0.001, C=0.50 
+ Fold1.Rep1: degree=1, scale=0.010, C=0.50 
- Fold1.Rep1: degree=1, scale=0.010, C=0.50 
+ Fold1.Rep1: degree=2, scale=0.010, C=0.50 
- Fold1.Rep1: degree=2, scale=0.010, C=0.50 
+ Fold2.Rep1: degree=1, scale=0.001, C=0.25 
- Fold2.Rep1: degree=1, scale=0.001, C=0.25 
+ Fold2.Rep1: degree=2, scale=0.001, C=0.25 
- Fold2.Rep1: degree=2, scale=0.001, C=0.25 
+ Fold2.Rep1: degree=1, scale=0.010, C=0.25 
- Fold2.Rep1: degree=1, scale=0.010, C=0.25 
+ Fold2.Re

Gastric_cancer,other
0.06831533,0.9316847
0.05985108,0.9401489
0.08731260,0.9126874
0.06429391,0.9357061
0.05269019,0.9473098
0.06309559,0.9369044
0.07248225,0.9275178
0.06354752,0.9364525
0.06763036,0.9323696
0.06363595,0.9363640


[1] 0.06831533 0.05985108 0.08731260 0.06429391 0.05269019 0.06309559
  [7] 0.07248225 0.06354752 0.06763036 0.06363595 0.05276487 0.04121024
 [13] 0.04840229 0.05229724 0.05250724 0.04621188 0.05497770 0.03763648
 [19] 0.05794943 0.05307516 0.06260534 0.03519670 0.06261156 0.06987271
 [25] 0.06671398 0.07024649 0.06417605 0.06385711 0.07038765 0.06707524
 [31] 0.05408754 0.06038015 0.03486147 0.05354686 0.05303090 0.06358764
 [37] 0.07065479 0.05693113 0.05581131 0.06773722 0.06570107 0.04100361
 [43] 0.05639629 0.03379526 0.06183894 0.05354509 0.05999258 0.05912482
 [49] 0.04754975 0.04177924 0.07447651 0.03945606 0.07030742 0.06471363
 [55] 0.05681684 0.05301471 0.06085675 0.05402727 0.06043293 0.06755988
 [61] 0.05716056 0.05143359 0.05633461 0.05146738 0.05916349 0.05814656
 [67] 0.05939831 0.06148559 0.05716497 0.05461400 0.05955033 0.05544577
 [73] 0.05410247 0.06449993 0.05510458 0.05781952 0.06257028 0.06616845
 [79] 0.05453743 0.05534298 0.04975011 0.05447542 0.06119853 0.05595071
 [85] 0.07402981 0.06638762 0.06264762 0.05195952 0.06006296 0.06352980
 [91] 0.05839694 0.05512038 0.06149605 0.06533057 0.06657507 0.06328787
 [97] 0.05617181 0.06346300 0.05519165 0.06633622 0.07541234 0.06301733
[103] 0.06965332 0.05752075 0.06538391 0.06542990 0.06895300 0.06535773
[109] 0.05117193 0.06737693 0.06001951 0.05862118 0.06038456 0.06885171
[115] 0.06133136 0.05955754 0.05718527 0.05637793 0.06207724 0.05832515
[121] 0.06816272 0.06561543 0.06131250 0.04878055 0.05995053 0.06175841
[127] 0.05926698 0.06195492 0.05081622 0.05800031 0.06554395 0.04985501
[133] 0.06249009 0.06274268 0.05068436 0.06805418 0.06021415 0.07095822
[139] 0.05019557 0.05410468 0.05905015 0.05912832 0.05824119 0.06192943
[145] 0.05645040 0.07307912 0.06663495 0.06126819 0.05757315 0.05898273
[151] 0.06967125 0.05560833 0.06682250 0.06995796 0.07694409 0.06267487
[157] 0.04858063 0.06693199 0.06284619 0.06934022 0.05824902 0.05420925
[163] 0.08962036 0.06673162 0.05531056 0.05290931 0.06680887 0.06513726
[169] 0.05593124 0.06305381 0.06711950 0.05787268 0.06774380 0.07252813
[175] 0.07144243 0.05190803 0.06045727 0.05675399 0.05016343 0.06038604
[181] 0.05433142 0.07748203 0.05001894 0.07194603 0.06278718 0.05998554
[187] 0.05431027 0.06693343 0.05653324 0.06581848 0.06220656 0.07073749
[193] 0.06373340 0.05750172 0.07583860 0.06814322 0.06555022 0.06572654
[199] 0.05042531 0.04523419 0.07343443 0.03741799 0.06344809 0.03917314
[205] 0.04411699 0.05229633 0.06210120 0.05667657 0.06753049 0.06284359
[211] 0.06733063 0.05888482

+ Fold1.Rep1: degree=1, scale=0.001, C=0.25 
- Fold1.Rep1: degree=1, scale=0.001, C=0.25 
+ Fold1.Rep1: degree=2, scale=0.001, C=0.25 
- Fold1.Rep1: degree=2, scale=0.001, C=0.25 
+ Fold1.Rep1: degree=1, scale=0.010, C=0.25 
- Fold1.Rep1: degree=1, scale=0.010, C=0.25 
+ Fold1.Rep1: degree=2, scale=0.010, C=0.25 
- Fold1.Rep1: degree=2, scale=0.010, C=0.25 
+ Fold1.Rep1: degree=1, scale=0.001, C=0.50 
- Fold1.Rep1: degree=1, scale=0.001, C=0.50 
+ Fold1.Rep1: degree=2, scale=0.001, C=0.50 
- Fold1.Rep1: degree=2, scale=0.001, C=0.50 
+ Fold1.Rep1: degree=1, scale=0.010, C=0.50 
- Fold1.Rep1: degree=1, scale=0.010, C=0.50 
+ Fold1.Rep1: degree=2, scale=0.010, C=0.50 
- Fold1.Rep1: degree=2, scale=0.010, C=0.50 
+ Fold2.Rep1: degree=1, scale=0.001, C=0.25 
- Fold2.Rep1: degree=1, scale=0.001, C=0.25 
+ Fold2.Rep1: degree=2, scale=0.001, C=0.25 
- Fold2.Rep1: degree=2, scale=0.001, C=0.25 
+ Fold2.Rep1: degree=1, scale=0.010, C=0.25 
- Fold2.Rep1: degree=1, scale=0.010, C=0.25 
+ Fold2.Re

Gastric_cancer,other
0.004151982,0.9958480
0.029375217,0.9706248
0.018955230,0.9810448
0.761963262,0.2380367
0.080040666,0.9199593
0.038624237,0.9613758
0.012680148,0.9873199
0.009890117,0.9901099
0.043246726,0.9567533
0.144452481,0.8555475


[1] 0.0041519823 0.0293752170 0.0189552295 0.7619632624 0.0800406664
  [6] 0.0386242374 0.0126801479 0.0098901173 0.0432467257 0.1444524811
 [11] 0.2676689022 0.5754752529 0.0505167038 0.1873523684 0.3756188484
 [16] 0.0404097655 0.0695010776 0.2719299202 0.0263923182 0.4425103260
 [21] 0.0190641330 0.0094717775 0.0223855390 0.0240513020 0.0146997432
 [26] 0.0679046594 0.0071378806 0.4539430970 0.0445756046 0.0472947258
 [31] 0.0280536228 0.0421498307 0.0156241113 0.0001926111 0.0770116924
 [36] 0.1242743857 0.0112519046 0.0133393639 0.0329839240 0.0113045434
 [41] 0.0061345779 0.0472076329 0.0164480155 0.0567875047 0.0676140854
 [46] 0.0062710885 0.4020661778 0.0002847621 0.0357559829 0.0935402045
 [51] 0.1002667802 0.0258317835 0.0082627812 0.0629553434 0.0229305627
 [56] 0.0420239166 0.0120551209 0.0526088978 0.0075041634 0.0375545458
 [61] 0.0823121963 0.0337120262 0.0465528696 0.0451371419 0.0249616019
 [66] 0.0227708719 0.1201559354 0.0265974337 0.0907266658 0.1064116727
 [71] 0.0352193935 0.0647672903 0.1255194742 0.0676332514 0.0081947585
 [76] 0.0167980473 0.0771783206 0.0401975077 0.0115430207 0.0707027288
 [81] 0.0742830195 0.2252253258 0.1241617016 0.0803652945 0.0209152315
 [86] 0.0267972408 0.0583355927 0.0300036023 0.1381182968 0.0847190908
 [91] 0.2697838970 0.0184722024 0.1419096898 0.1396177142 0.0020000208
 [96] 0.1027706415 0.1928925606 0.1287534740 0.0074423177 0.0155160606
[101] 0.0221832610 0.0203456569 0.1572360136 0.0308635560 0.0266760937
[106] 0.0421931155 0.0065059456 0.0574309335 0.0305011548 0.0068444160
[111] 0.0302580183 0.0409344172 0.0808858952 0.0284835173 0.0635777984
[116] 0.0348650358 0.0918791403 0.0683802655 0.0204442213 0.1304092629
[121] 0.0321776967 0.0269455731 0.0455509187 0.0526962204 0.0256672333
[126] 0.2311410290 0.1113833244 0.0891659595 0.0765008337 0.0373889886
[131] 0.0828768087 0.0510480269 0.1980367335 0.0242939852 0.4406851480
[136] 0.0281899361 0.1740049314 0.1150711182 0.1024679054 0.0870099161
[141] 0.0804811715 0.0203253780 0.0368669241 0.0885200566 0.0063700738
[146] 0.0614449156 0.0139825779 0.0070718595 0.0160764619 0.5264187723
[151] 0.0226675201 0.0103720871 0.0140111043 0.0039251427 0.0418543368
[156] 0.6234254934 0.0723828216 0.0209938102 0.0031865999 0.0327498161
[161] 0.2095706324 0.1422305414 0.0809800024 0.0091435295 0.0072806568
[166] 0.0515863583 0.0635714052 0.0045614776 0.0426831063 0.0497922133
[171] 0.0395819948 0.0337893328 0.2340333438 0.0561984110 0.0087372078
[176] 0.0062625934 0.0326298226 0.0177947471 0.0510621740 0.0129969354
[181] 0.0196144798 0.4622575292 0.0759628507 0.1120153412 0.0095091058
[186] 0.0081109234 0.0574304708 0.0024398292 0.0698566332 0.0237343337
[191] 0.0380543168 0.0188250803 0.0420769388 0.0055100170 0.0237218785
[196] 0.0201462187 0.0086794817 0.0424487305 0.0026595419 0.0071971591
[201] 0.1669929659 0.0891225963 0.5067802404 0.0576470405 0.0169720560
[206] 0.2250391070 0.4291837658 0.0668262013 0.0002015926 0.0311508887
[211] 0.0087103161

In [ ]:
head(results)

In [ ]:
for ( col in 1:ncol(results)){
    colnames(results)[col] <-  sub("_pred", "", colnames(results)[col])
}
head(results)

In [ ]:
saveRDS(results, file = snakemake@output[["SVM_output"]])

## AUC calculation with pROC package == One vs. one

In [ ]:
rocs <- list()
list_names <- paste0("CV_rep_", 1:k_outer_cv)
for (i in 1:k_outer_cv){
    res_CV <- results %>% filter(CV_rep == i) %>% select(-c(CV_rep, observed))
    res_CV <- 
    roc <- multiclass.roc(response = data$sample_type, predictor = res_CV)
    rocs[[i]] <- roc
}
names(rocs) <- c(list_names)

paste0("The AUC of the first CV repetition: ", rocs[[1]]$auc)

aucs <- c()
for (i in 1:length(rocs)){
    aucs <- c(aucs, rocs[[i]]$auc) 
}

paste0("The mean AUC of all the CV repetitions: ", mean(aucs))

# AUC calculation with multiROC package == One vs. rest, and plotting

In [ ]:
Specificities <- NULL
Sensitivities <- NULL

for (i in 1:k_outer_cv){
    res_CV <- data.frame(results %>% filter(CV_rep == i) %>% select(-c(CV_rep, observed)))
    
    colnames(res_CV) <- paste(colnames(res_CV), "_pred_lasso", sep = "")

    true_label <- dummies::dummy(data$sample_type, sep = ".")
    true_label <- data.frame(true_label)
    colnames(true_label) <- gsub(".*?\\.", "", colnames(true_label))
    colnames(true_label) <- paste(colnames(true_label), "_true", sep = "")
    final_df <- cbind(true_label, res_CV)

    roc_res <- multi_roc(final_df, force_diag=T)
    
    if (i == 1){
        AUCs <- as.data.frame(t(unlist(roc_res$AUC)))
    }
    else {
        AUCs <- rbind(AUCs, as.data.frame(t(unlist(roc_res$AUC))))
    }
    
    plot_roc_df <- plot_roc_data(roc_res)
    plot_roc_df <- plot_roc_df %>% select(-Method)
    
    if (i == 1){
        Specificities <- plot_roc_df %>% select(Specificity)
        Sensitivities <- plot_roc_df %>% select(Sensitivity)
        Groups  <- plot_roc_df %>% select(Group)
    }
    else {
        Specificities <- cbind(Specificities, plot_roc_df %>% select(Specificity))
        Sensitivities <- cbind(Sensitivities, plot_roc_df %>% select(Sensitivity))
    }
    
    #IRdisplay::display(all_plot_roc_df)

}
Specificities <- tibble(Specificities, .name_repair = "unique")
Specificities <- Specificities %>% mutate(mean = rowMeans(across()))
Sensitivities <- tibble(Sensitivities, .name_repair = "unique")
Sensitivities <- Sensitivities %>% mutate(mean = rowMeans(across()))

(AUCs <- tibble(AUCs))

all_sensitivity_specificity <- tibble(Specificity = Specificities$mean, Sensitivity = Sensitivities$mean, Group = Groups$Group)
head(all_sensitivity_specificity)

In [ ]:
ggplot(all_sensitivity_specificity, aes(x = 1-Specificity, y=Sensitivity)) +
  geom_path(aes(color = Group), size=1) +
  geom_segment(aes(x = 0, y = 0, xend = 1, yend = 1), 
                        colour='grey', linetype = 'dotdash') +
  theme_bw() + 
  theme(plot.title = element_text(hjust = 0.5), 
                 legend.justification=c(1, 0), legend.position=c(.95, .05),
                 legend.title=element_blank(), 
                 legend.background = element_rect(fill=NULL, size=0.5, 
                                                           linetype="solid", colour ="black"))

In [ ]:
print("Mean AUC over 10 repetitions of 10-fold CV: ")
(mean_AUCs <- colMeans(AUCs[sapply(AUCs, is.numeric)]))

In [ ]:
results01 <- tibble()

for (i in 1:k_outer_cv){
    res_CV <- results %>% filter(CV_rep == i)
    obs_rep <- res_CV %>% select(c(CV_rep, observed))
    res_CV <- res_CV %>% select(-c(CV_rep, observed))
    res_CV <- res_CV %>% mutate(pred01 = factor(colnames(res_CV)[apply(res_CV,1,which.max)], ordered = TRUE))
    res_CV <- cbind(res_CV, obs_rep)
    results01 <- rbind(results01, res_CV)
}

head(results01)

In [ ]:
error_rates <- c()
accuracies <- c()
for (i in 1:k_outer_cv){
    res_CV <- results01 %>% filter(CV_rep == i)
    error_rates <- c(error_rates, mean(as.character(res_CV$observed) != as.character(res_CV$pred01)))
    accuracies <- c(accuracies, mean(as.character(res_CV$observed) == as.character(res_CV$pred01)))
}

error_rates
accuracies

paste("Mean error rate over 10 repetitions of 10-fold CV: ", mean(error_rates), sep = "")
paste("Mean accuracy over 10 repetitions of 10-fold CV: ", mean(accuracies), sep = "")

### for one vs rest classifier: https://stats.stackexchange.com/questions/71700/how-to-draw-roc-curve-with-three-response-variable/110550#110550

In [ ]:
install.packages("ROCR")
library(ROCR)
install.packages("rlist")
library(rlist)

In [ ]:
list_res <- list(predictions = list(), labels = list())
str(list_res)

In [ ]:
classes <- unique(data$sample_type)

aucs = c()

plot(x=NA, y=NA, xlim=c(0,1), ylim=c(0,1),
     bty='n')
legend_text <- c()

for (i in 1:length(classes)) {
    for (rep in 1:k_outer_cv){
        res_CV <- results %>% filter(CV_rep == rep) %>% select(-c(observed, CV_rep)) 
        class <- paste0(classes[i], "")
        pred = pull(res_CV, class)
        list.append(list_res$predictions, rep = pred)
        obs <- ifelse(data$sample_type == class, 1, 0)
        list.append(list_res$labels, rep = obs)
        }
    pred = prediction(pred, obs)
    nbperf = performance(pred, "tpr", "fpr")
    
    roc.x = unlist(nbperf@x.values)
    roc.y = unlist(nbperf@y.values)
    par(new=TRUE)
    #lines(roc.y ~ roc.x, col=i+1, lwd=2)
    plot(nbperf,
     avg='threshold',
     lwd=3,
     main='Horizontal averaging',
     col=i + 1)
    legend_text <- c(legend_text, class)


    nbauc = performance(pred, "auc")
    nbauc = unlist(slot(nbauc, "y.values"))
    aucs <- c(aucs, nbauc)
    }

lines(x=c(0,1), c(0,1))
mean(aucs)
aucs
legend('bottomright', legend = legend_text, lty = 1, col=2:(length(classes)+1))